# Optimization problem

m3cUlabs has developed a new vaccine aggainst COVID-19, but it has only been aproved by the Junta de Andalucia. Therefore they can only supply vaccines to each of the 8 provinces of Andalucia. They have done several studies and have developed through inference the porcentage of people that will be inmune once vaccinated given the province and the day of the vaccination campaign. 

This laboratory knows that his vaccine is not the best one developed yet. However they do not want everybody to know it. Therefore they want to carefully supply vaccines to those provinces that will have higher rates of inmunity.

All the same, Junta de Andalucia knows how pharmaceutical operates and their aim to obtain benefits and maintain a good reputation. So they havw impossed some constraints about the supply of the vaccines. The units will be thousand of vaccines. There will be needed at least:

- At least 50 vaccines per province and day
- No more than 200 vaccines per province and day
- At least 2000 vaccines per province within the whole campaing

Nevertheless m3cUlabs has also their restrictions. They will not be able to supply more than between 900 and 1100 thousand of vaccines a day.

The objective of m3cUlabs is to maximize the imunization of the population so they can get a good reputation and expand their bussiness to other parts of Spain.

## Create dataset

As I have created the problem, I also have to create the dataset with the random probabilities of inmunization per province. For that we will create a matrix and use the random funciton provided by numpy. These will be the parameters of out optimization problem.

In [115]:
import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)
supply=np.empty(20)
i=0
while(i<20):
    supply[i]=np.random.randint(900,1100)
    i+=1
supply = supply.astype(int)

Once we have the matrix with the weights we can create a dataframe to show it more clearly. 

In [56]:
import pandas as pd
data=pd.DataFrame(X)
data.rename(index = {0 : 'Malaga', 1:"Sevilla",2:"Cordoba",3:"Cadiz",4:"Huelva",5:"Jaen",6:"Granada",7:"Almeria"}, inplace = True)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Malaga,0.42,0.72,0.00,0.30,0.15,0.09,0.19,0.35,0.40,0.54,0.42,0.69,0.20,0.88,0.03,0.67,0.42,0.56,0.14,0.20
Sevilla,0.80,0.97,0.31,0.69,0.88,0.89,0.09,0.04,0.17,0.88,0.10,0.42,0.96,0.53,0.69,0.32,0.69,0.83,0.02,0.75
Cordoba,0.99,0.75,0.28,0.79,0.10,0.45,0.91,0.29,0.29,0.13,0.02,0.68,0.21,0.27,0.49,0.05,0.57,0.15,0.59,0.70
Cadiz,0.10,0.41,0.69,0.41,0.05,0.54,0.66,0.51,0.94,0.59,0.90,0.14,0.14,0.81,0.40,0.17,0.93,0.35,0.75,0.73
Huelva,0.88,0.62,0.75,0.35,0.27,0.90,0.43,0.96,0.66,0.62,0.11,0.95,0.45,0.58,0.41,0.24,0.90,0.57,0.00,0.62
Jaen,0.33,0.53,0.89,0.36,0.91,0.62,0.02,0.93,0.69,1.00,0.17,0.14,0.93,0.70,0.07,0.76,0.75,0.92,0.71,0.12
Granada,0.02,0.03,0.03,0.25,0.86,0.54,0.55,0.84,0.12,0.28,0.59,0.97,0.56,0.02,0.80,0.23,0.81,0.39,0.86,0.75
Almeria,0.56,0.14,0.06,0.12,0.04,0.11,0.23,0.71,0.56,0.01,0.07,0.97,0.57,0.20,0.25,0.74,0.20,0.58,0.97,0.85


The supply per day:

In [120]:
data_supply=pd.DataFrame(supply)
data_supply.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1088,934,1000,932,1047,1095,924,994,1066,1075,1033,979,963,1013,987,1060,942,1094,1058,1045


## Formulate mathematically the problem as a linear optimization model

The main objective of the project is to maximize the number of inmune people in Andalucía in the whole campaign. The variables in this problem will be the number of vaccines sent to each province, which multiplied by the porcentage will give the number of people inmunized. Therefore, our objective function will be:

\begin{align*}
  \text{maximize} \sum_{i=0}^7 \sum_{j=0}^{19} M_{ij}x_{ij}
\end{align*}

Subject to:

\begin{align*}
   x_{ij} \geq 50 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 
\end{align*}

\begin{align*}
   x_{ij} \leq 200 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 
\end{align*}

\begin{align*}
   \sum_{j=0}^{19} x_{ij} \geq 2000 , \hspace{0.5cm} i=0,...,7
\end{align*}

\begin{align*}
   \sum_{i=0}^{7} x_{ij} = s_{j} , \hspace{0.5cm} j=0,...,19
\end{align*}


##  Implement the model in Pyomo as a general model and solve it

I will now develop a programme in pyomo to solve the optimzation problem that I have proposed above.

In [137]:
%%writefile test.py
from pyomo.environ import *

import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)

supplyer=np.empty(20)
i=0
while(i<20):
    supplyer[i]=np.random.randint(900,1100)
    i+=1
supplyer = supplyer.astype(int)

model = ConcreteModel()

model.I = RangeSet(0,X.shape[0]-1) #Cities
model.J = RangeSet(0,X.shape[1]-1) #Days

#variable
model.x = Var(model.I,model.J, domain=NonNegativeReals)

def Obj_rule(model):
	return sum(sum( model.x[i,j]*X[i,j] for i in model.I) for j in model.J)

model.Obj = Objective(rule=Obj_rule, sense=maximize)

#constraint min vaccines per day and city
def min_vac_day_city(model, i,j): 
	return (model.x[i,j] >= 50) 
model.min_vac_day_city = Constraint(model.I, model.J,rule=min_vac_day_city)

#constraint max vaccines per day and city
def max_vac_day_city(model, i,j): 
	return (model.x[i,j] <= 200) 
model.max_vac_day_city = Constraint(model.I, model.J,rule=max_vac_day_city)

#Min per city
def min_city_all(model, i): 
	return sum(model.x[i,j] for j in model.J) >= 2000  
model.min_city_all = Constraint(model.I, rule=min_city_all)

#Supply
def supply(model, j): 
	return sum(model.x[i,j] for i in model.I) == supplyer[j]  
model.supply = Constraint(model.J, rule=supply)



Overwriting test.py


In [138]:
!pyomo solve test.py --solver=glpk --summary

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.23] Creating model
[    0.23] Applying solver
[    0.27] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 12721.98
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=160, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 0) :     0 : 138.0 :  None : False : False : NonNegativeReals
         (0, 1) :     0 : 200.0 :  None : False : False : NonNegativeReals
         (0, 2) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 3) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 4) :     0 :  97.0 :  None : False : False : NonNegativeReals
         (0, 5) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 6) :     0 :  50.0 :  None : False : False : NonNegativeReals
  

The output obtained shows the solution generated by pyomo for maximizing the objective function defined. However, as the solution is quite large it is difficult to completely check it. Therefore, we will take a day and see if the constrains are satisfied or not.

For the first day, that is day 0 we obtained the following results for each province:

- Malaga -> 138
- Sevilla -> 200
- Cordoba -> 200
- Cadiz -> 50
- Huelva -> 200
- Jaen -> 50
- Granada -> 50
- Almeria -> 200

1.  Max vaccines per day and city: We will check thatthere are no provinces with more than 200 vaccines sent in that day. Indeed we can see that the maximum sent is 200.

2. Min vaccine sper day and city: We set that each city needs at least 50 vaccines a days. We can see in the output that there are no days below that bound.

3. Maximum supply of vaccines per day is equal to the supply fixed. The first day the supply was set at 1088. The sum of all the vaccines received for each problem is indeed 1088.

4. Each city mush have at least 2000 vaccines received in the whole campaign. For checking this constraint we will look at the results obtained for Malaga:

- (0, 0) :138.0  
- (0, 1) :200.0  
- (0, 2) :50.0  
- (0, 3) :50.0  
- (0, 4) :97.0  
- (0, 5) :50.0  
- (0, 6) :50.0 
- (0, 7) :50.0 
- (0, 8) :116.0 
- (0, 9) :125.0  
- (0, 10) :200.0  
- (0, 11) :50.0 
- (0, 12) :50.0 
- (0, 13) :200.0   
- (0, 14) :50.0  
- (0, 15) :200.0 
- (0, 16) :50.0 
- (0, 17) :174.0 
- (0, 18) :50.0 
- (0, 19) :50.0

As it can be seen, the sum of all these results are indeed 2000. From this we can conclude that Málaga has not in general high rates for inmunization. Therefore the constraint is satisfied at the minimum.

As all the constraints are satisfied, we can say that the optimzation problem is correct.


## Modify the problem in a) to impose some logical or conditional (linear) constraints that require the use of binary or integer variables.

For this case I have to reformualte the problem that we will be optimizing to include logical conditions. So far we have been trying to maximize the number of inmune people per province, having a limited supply number of vaccines and some other constraints.

From now on we will still be trying to maximize the inmunization of the population. However, we are going to fix the units of vaccines that we will sent to each province. That is, each province will receive 8000 vaccines if selected that day, but they will not be able to receive two days in a row. Therefore, our new function to maximize is:

Since the problem is different, we also have to modify the constraints. Now we wont have a minimum demand per province and day as before. Neither the minimum number of vaccines per province in the campaing and the constraint leveling the supply. 

However, we have to include some new variables now. There can only be three provinces chosen to send vaccines to each day. Two provinces cannot receive vaccines two days in a row and the minimum number of vaccines per province at the end of the campaign should be higher than 32000.

Therefore, our new optimization problem is:

\begin{align*}
  \text{maximize} \sum_{i=0}^7 \sum_{j=0}^{19} 8000M_{ij}x_{ij}
\end{align*}


Subject to:

\begin{align*}
   \sum_{i=0}^{7} x_{ij} <= 3 , \hspace{0.5cm} j=0,...,19
\end{align*}

\begin{align*}
   \sum_{j=0}^{19} 8000x_{ij} \geq 32000 , \hspace{0.5cm} i=0,...,7
\end{align*}

\begin{align*}
    x_{ij-1} + x_{i,j} \leq 1 \hspace{0.5cm} \forall j=0,...,18 \in \hspace{0.5cm} i=0,...,7
\end{align*}

We implement and solve it using pyomo in the following chunk of code.

In [175]:
%%writefile test2.py
from pyomo.environ import *

import numpy as np
import pandas as pd

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)


model = ConcreteModel()

model.I = RangeSet(0,X.shape[0]-1) #Provinces
model.J = RangeSet(0,X.shape[1]-1) #Days

#variable
model.x = Var(model.I,model.J, domain=NonNegativeReals, bounds=(0,1))

def Obj_rule(model):
	return sum(sum( 8000*model.x[i,j]*X[i,j] for i in model.I) for j in model.J)

model.Obj = Objective(rule=Obj_rule, sense=maximize)

#Send vaccines to maximum 3 provinces per day
def max_vaccines_province(model,j):
    return(sum(model.x[i,j] for i in model.I)<=3)
model.max_vaccines_provinve=Constraint(model.J,rule=max_vaccines_province)

#Min per city
def min_city_binary(model, i): 
	return sum(model.x[i,j]*8000 for j in model.J) >= 32000  
model.min_city_binay = Constraint(model.I, rule=min_city_binary)

#No receiving vaccines two days in a row
def no_two_row(model,i,j):
    if (j!=19):
        return (model.x[i,j] + model.x[i,j+1] <=1)
    else:
        return (model.x[i,j] + model.x[i,j-1] <= 1)
model.no_two_row=Constraint(model.I,model.J,rule=no_two_row)


Overwriting test2.py


In [176]:
!pyomo solve test2.py --solver=glpk --summary

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.24] Creating model
[    0.24] Applying solver
[    0.28] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 351440.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=160, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 0) :     0 :   0.0 :     1 : False : False : NonNegativeReals
         (0, 1) :     0 :   1.0 :     1 : False : False : NonNegativeReals
         (0, 2) :     0 :   0.0 :     1 : False : False : NonNegativeReals
         (0, 3) :     0 :   1.0 :     1 : False : False : NonNegativeReals
         (0, 4) :     0 :   0.0 :     1 : False : False : NonNegativeReals
         (0, 5) :     0 :   1.0 :     1 : False : False : NonNegativeReals
         (0, 6) :     0 :   0.0 :     1 : False : False : NonNegativeReals
  

I will now check that the results obtained are aligned with the restrictions for different examples:

 - At least 32000 vaccines at the end of the campaign: Analyzing the output we can see that Malaga received vaccines on days [1,3,5,8,10,13,15,17], that it 8 days receiving 8000 vaccines each days. Wich in total brings 64000 vaccines in the total campaign, which satisfies the constraint.

 - For each day, no more than 3 provinces can receive vaccines: Analyzing day 0 and 1 we can see that on day 0 only provinces [4,5,7] were selected, whereas in the second day [0,1,2] received vaccines.

 - No province can receive vaccines two days in a row. Analyzing for example Cadiz, we can see that it received vaccines on days [2,4,6,8,10,13,16,18], which indeed satisfies the constraint impossed.

## d) Assume now that one or some of the model parameters in a) or c) are uncertain, and decisions need to be made facing this uncertainty.

### i) (2 points) Propose a stochastic programming formulation for the problem presented in a) or c) (or an adapted version of them): Identify the first- and second-stage decision variables, generate an adequate set of random scenarios for the uncertain parameters and solve the resulting model in Pyomo-Python. Interpret the results.

Until this section we have been modeling a problem with the total supply per day fixed. However the daily production of vaccines is not possible to set as it depends on several factors. Therefore, to face this problem we will assume that the total supply is uncertain. To solve this problem we will present an stochastic programming formulation.

We will have then two different possible cases for the supply, whether a machine in the laboratory breaks during the campaign or not. Depending on that we will have a supply varying between [1200,1000] and [1000,800] for having a problem with any machine in the laboratory or not respectively.

Also, the constraints in our problem do not vary at all. The only change would be that for now we have to introduce the different cases possible for the supply.

Therefore the formualtion of the problem will stay pretty much the same. The difference will be again the introduction of an index k representing the different cases available: 0 (Initial problem), 1(best case), 2(problems with machines). 




\begin{align*}
  \text{maximize} \sum_{i=0}^7 \sum_{j=0}^{19} M_{ij}x_{ij} + \sum_{k=0}^\omega \pi_{k}(\sum_{i=0}^7 \sum_{j=0}^{19} M_{ij}x_{kij})
\end{align*}

Subject to:

\begin{align*}
   x_{kij} \geq 50 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 \hspace{0.5cm} k=0,1,2
\end{align*}

\begin{align*}
   x_{kij} \leq 200 , \hspace{0.5cm} i=0,...,7, \hspace{0.2cm} j = 0,...,19 \hspace{0.5cm} k=0,1,2
\end{align*}

\begin{align*}
   \sum_{j=0}^{19} x_{kij} \geq 2000 , \hspace{0.5cm} i=0,...,7 \hspace{0.5cm} k=0,1,2
\end{align*}

\begin{align*}
   \sum_{i=0}^{7} x_{kij} = s_{kj} , \hspace{0.5cm} j=0,...,19 \hspace{0.5cm} k=0,1,2
\end{align*}

In [239]:
%%writefile test3.py
from pyomo.environ import *
opt = SolverFactory("glpk")
from pyomo.opt import SolverFactory
import numpy as np

modelcv = AbstractModel()

X=np.empty([8,20])
np.asmatrix(X)
np.random.seed(1)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        X[i,j]=np.random.random()

X=np.round(X,2)

supplyer=np.empty(20)
i=0
while(i<20):
    supplyer[i]=np.random.randint(900,1100)
    i+=1
supplyer = supplyer.astype(int)

supplyer_c=np.empty([3,20])
np.asmatrix(supplyer_c)
i=0
j=0
while(j<3):
    if(j==0):
        for i in range(0,20):
            supplyer_c[j,i]=np.random.randint(900,1100)
            i+=1
    elif(j==1):
        for i in range(0,20):
            supplyer_c[j,i]=np.random.randint(1000,1200)
            i+=1
    else:
        for i in range(0,20):
            supplyer_c[j,i]=np.random.randint(800,1000)
            i+=1
    j+=1

supplyer_c = supplyer_c.astype(int)
supplyer_c

n_sc=2

modelcv.k = RangeSet(0,n_sc-1)#scenarios
modelcv.I = RangeSet(0,X.shape[0]-1) #Provinces
modelcv.J = RangeSet(0,X.shape[1]-1) #Day

pi=np.array([0.4,0.6])

#model variables: continuous
modelcv.Q = Var(modelcv.I,modelcv.J,domain=NonNegativeReals) 
modelcv.Qc = Var(modelcv.k,modelcv.I,modelcv.J,domain=NonNegativeReals) 

def Obj_rule_cv(modelcv):
	return (sum(sum( modelcv.Q[i,j]*X[i,j] for i in modelcv.I) for j in modelcv.J)) + (sum(pi[k]*(sum(sum( modelcv.Qc[k,i,j]*X[i,j] for i in modelcv.I)for j in modelcv.J)) for k in modelcv.k))
modelcv.Obj = Objective(rule=Obj_rule_cv, sense=maximize)

#constraint min vaccines per day and city
def min_vac_day_city_Q(modelcv, i,j): 
	return (modelcv.Q[i,j] >= 50) 
modelcv.min_vac_day_city_Q = Constraint(modelcv.I, modelcv.J,rule=min_vac_day_city_Q)

def min_vac_day_city_Qc(modelcv, k,i,j): 
	return (modelcv.Qc[k,i,j] >= 50) 
modelcv.min_vac_day_city_Qc = Constraint(modelcv.k,modelcv.I, modelcv.J,rule=min_vac_day_city_Qc)

#constraint max vaccines per day and city
def max_vac_day_city_Q(modelcv, i,j): 
 	return (modelcv.Q[i,j] <= 200) 
modelcv.max_vac_day_city_Q = Constraint(modelcv.I, modelcv.J,rule=max_vac_day_city_Q)

#constraint max vaccines per day and city
def max_vac_day_city_Qc(modelcv, i,j,k): 
 	return (modelcv.Qc[k,i,j] <= 200) 
modelcv.max_vac_day_city_Qc = Constraint(modelcv.I, modelcv.J,modelcv.k,rule=max_vac_day_city_Qc)

#Min per city
def min_city_all_Q(modelcv, i): 
 	return sum(modelcv.Q[i,j] for j in modelcv.J) >= 2000  
modelcv.min_city_all_Q = Constraint(modelcv.I, rule=min_city_all_Q)

def min_city_all_Qc(modelcv, i,k): 
 	return sum(modelcv.Qc[k,i,j] for j in modelcv.J) >= 2000  
modelcv.min_city_all_Qc = Constraint(modelcv.I,modelcv.k, rule=min_city_all_Qc)

#Supply
def supply_Q(modelcv, j): 
 	return sum(modelcv.Q[i,j] for i in modelcv.I) == supplyer_c[0,j]  
modelcv.supply_Q = Constraint(modelcv.J, rule=supply_Q)

def supply_Qc(modelcv, j,k): 
 	return sum(modelcv.Qc[k,i,j] for i in modelcv.I) == supplyer_c[k+1,j]  
modelcv.supply_Qc = Constraint(modelcv.J, modelcv.k,rule=supply_Qc)


Overwriting test3.py


In [240]:
!pyomo solve test3.py --solver=glpk --summary

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.09] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 25127.73
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    Q : Size=160, Index=Q_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 0) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 1) :     0 : 200.0 :  None : False : False : NonNegativeReals
         (0, 2) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 3) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 4) :     0 : 117.0 :  None : False : False : NonNegativeReals
         (0, 5) :     0 :  50.0 :  None : False : False : NonNegativeReals
         (0, 6) :     0 :  50.0 :  None : False : False : NonNegativeReals
  

### ii) (1 points) Compute the Expected Value of Perfect Information (EVPI) and the Value of the Stochastic Solution (VSS) associated to the optimal solution obtained in i).